Main Caption Function

In [1]:
import subprocess
import os

# Specify the folder containing the WAV files
wav_folder = 'D:/AfterEffectsBot/Assets/audios'  # Replace with your actual path

# List all files in the folder
wav_files = [file for file in os.listdir(wav_folder) if file.startswith('youtube_audio') and file.endswith('.wav')]

# Loop through each WAV file in the folder
for wav_file in wav_files:
    command = f"whisperx {os.path.join(wav_folder, wav_file)} --model medium.en --align_model WAV2VEC2_ASR_LARGE_LV60K_960H"
    
    # Execute the command
    subprocess.run(command, shell=True)


JSON To Word TO word Srt

In [2]:
import json
import os

def seconds_to_srt_time(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{milliseconds:03d}"

# Specify the folder containing the JSON files
json_folder = 'D:\Youtube Bot Audio'  # Replace with your actual path

# Specify the destination folder for the SRT files
output_folder = 'D:\Youtube Bot Audio'  # Replace with your desired output path

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all JSON files in the folder
json_files = [file for file in os.listdir(json_folder) if file.startswith('youtube_audio') and file.endswith('.json')]

for json_file in json_files:
    input_json_file = os.path.join(json_folder, json_file)
    output_file_name = f'caption_of_{json_file[:-5]}.srt'
    output_file_path = os.path.join(output_folder, output_file_name)

    with open(input_json_file, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)

    segments = data["segments"]

    with open(output_file_path, "w", encoding="utf-8") as srt_file:
        word_number = 1
        for segment in segments:
            start_time = segment["start"]
            for word in segment["words"]:
                # Check if the "end" key exists in the word dictionary
                if "end" in word:
                    end_time = word["end"]
                else:
                    # If "end" key is missing, assume a small duration (e.g., 0.1 seconds)
                    end_time = start_time + 0.1
                text = word["word"]
                srt_file.write(f"{word_number}\n")
                srt_file.write(f"{seconds_to_srt_time(start_time)} --> {seconds_to_srt_time(end_time)}\n")
                srt_file.write(f"{text}\n\n")
                word_number += 1
                start_time = end_time  # Update start time for the next word

    print(f"SRT file '{output_file_name}' with corrected timestamps has been generated successfully in the '{output_folder}' folder.")


SRT file 'caption_of_youtube_audio1.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.
SRT file 'caption_of_youtube_audio2.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.
SRT file 'caption_of_youtube_audio3.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.
SRT file 'caption_of_youtube_audio4.srt' with corrected timestamps has been generated successfully in the 'D:\Youtube Bot Audio' folder.


90 lettors

In [3]:
import os
import glob

def combine_words_to_caption(input_srt_file, output_srt_file):
    with open(input_srt_file, 'r', encoding='utf-8') as file:
        srt_content = file.read().splitlines()

    combined_srt = []
    words = []
    total_character_count = 0
    timestamps = []
    previous_caption = ""

    def add_caption():
        nonlocal combined_srt, timestamps, words, previous_caption
        if timestamps and len(timestamps) > 0:
            caption_text = " ".join(words)
            if caption_text != previous_caption:
                combined_srt.append(str(len(combined_srt) // 4 + 1))  # Calculate caption number
                if len(timestamps) == 1:
                    combined_srt.append(timestamps[0])
                else:
                    start_time, end_time = timestamps[0].split(" --> ")
                    combined_srt.append(f"{start_time} --> {end_time}")
                combined_srt.append(caption_text)
                combined_srt.append("")
                previous_caption = caption_text
        timestamps = []

    for i, line in enumerate(srt_content):
        line = line.strip()
        if not line:
            continue
        if line.isdigit() and (i == 0 or line != srt_content[i - 1]):
            # Check if it's a non-repeated numeric line
            add_caption()
        elif "-->" in line:
            timestamps.append(line)
        else:
            words_in_line = line.split()
            for word in words_in_line:
                word_len = len(word)
                if total_character_count + word_len + len(words) + 1 <= 90:
                    words.append(word)
                    total_character_count += word_len
                else:
                    # If the word is too long to fit, add the current caption and start a new one
                    add_caption()
                    if word_len > 90:
                        # If the word itself is too long, split it into smaller segments
                        words = [word[i:i + 90] for i in range(0, len(word), 90)]
                    else:
                        words = [word]
                    total_character_count = word_len

    add_caption()

    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.write("\n".join(combined_srt))

if __name__ == '__main__':
    input_files = glob.glob('caption_of_youtube_audio*.srt')
    for input_srt_file in input_files:
        output_srt_file = f'output_combined_with_timestamps_{os.path.basename(input_srt_file)}'
        combine_words_to_caption(input_srt_file, output_srt_file)


dublicate captions

In [4]:
import glob
import os

def duplicate_two_word_captions(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as input_srt:
        srt_content = input_srt.read()

    lines = srt_content.split('\n')
    output_lines = []
    caption_number = 1
    previous_line_was_two_words = False
    current_line_number = 0  # Track the current line number
    timestamp_line = ""

    for line in lines:
        current_line_number += 1  # Increment the current line number
        output_lines.append(line)  # Add the original line to the output

        if line.isdigit():  # Check if it's a caption number
            caption_number = int(line)
            previous_line_was_two_words = False
        elif '-->' in line:  # Check if it's a timestamp line
            previous_line_was_two_words = False
            timestamp_line = line
        elif line.strip():  # Check if it's a non-empty caption line
            words = line.split()
            if len(words) == 2:  # Check if there are exactly 2 words
                if previous_line_was_two_words:
                    # Insert a blank line and duplicate the caption three lines above the current position
                    output_lines.insert(current_line_number - 2, '')
                    output_lines.append(line)
                previous_line_was_two_words = True
                # Duplicate the caption with the same timestamp and caption number
                output_lines.append(f"{caption_number}\n{timestamp_line}\n{line}")
            else:
                previous_line_was_two_words = False

    with open(output_file, 'w', encoding='utf-8') as output_srt:
        output_srt.write('\n'.join(output_lines))

if __name__ == '__main__':
    input_files = glob.glob('output_combined_with_timestamps_caption_of_youtube_audio*.srt')

    for input_file in input_files:
        output_file = f'duplicated_{os.path.basename(input_file)}'
        duplicate_two_word_captions(input_file, output_file)
        print(f"Duplicated two-word captions in {input_file} and saved the result to {output_file}")


Duplicated two-word captions in output_combined_with_timestamps_caption_of_youtube_audio1.srt and saved the result to duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt
Duplicated two-word captions in output_combined_with_timestamps_caption_of_youtube_audio2.srt and saved the result to duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt
Duplicated two-word captions in output_combined_with_timestamps_caption_of_youtube_audio3.srt and saved the result to duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt
Duplicated two-word captions in output_combined_with_timestamps_caption_of_youtube_audio4.srt and saved the result to duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt


removing spaces

In [5]:
import glob

# Use glob to find all output files from the previous script
input_files = glob.glob('duplicated_output_combined_with_timestamps_caption_of_youtube_audio*.srt')

for input_file_name in input_files:
    # Create a new output file name based on the input file name
    output_file_name = f'filtered_{input_file_name}'

    # Open the input and output SRT files
    with open(input_file_name, 'r', encoding='utf-8') as input_file, open(output_file_name, 'w', encoding='utf-8') as output_file:
        for line in input_file:
            if not line.isspace():  # Check if the line is not empty or contains only whitespace
                output_file.write(line)

    print(f"Filtered the input file {input_file_name} and saved the result to {output_file_name}")


Filtered the input file duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt and saved the result to filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt
Filtered the input file duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt and saved the result to filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt
Filtered the input file duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt and saved the result to filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt
Filtered the input file duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt and saved the result to filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt


removing exter stuff first dub caption

In [6]:
import glob

# Lines to remove (e.g., lines 4, 5, and 6)
lines_to_remove = [4, 5, 6]

# Use glob to find all filtered output files from the previous script
input_files = glob.glob('filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio*.srt')

for input_file_name in input_files:
    # Create a new output file name based on the input file name
    output_file_name = f'removed_{input_file_name}'

    # Open the input and output SRT files
    with open(input_file_name, 'r', encoding='utf-8') as input_file, open(output_file_name, 'w', encoding='utf-8') as output_file:
        line_number = 0

        for line in input_file:
            line_number += 1

            # Check if the current line number is not in the list of lines to remove
            if line_number not in lines_to_remove:
                output_file.write(line)

    print(f"Removed specified lines from {input_file_name} and saved the result to {output_file_name}")


Removed specified lines from filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt and saved the result to removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt
Removed specified lines from filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt and saved the result to removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt
Removed specified lines from filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt and saved the result to removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt
Removed specified lines from filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt and saved the result to removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt


adding spaces

In [7]:
import glob

# Use glob to find all "removed" output files from the previous script
input_files = glob.glob('removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio*.srt')

for input_file_name in input_files:
    # Create a new output file name based on the input file name
    output_file_name = f'inserted_{input_file_name}'

    # Open the input and output SRT files
    with open(input_file_name, 'r', encoding='utf-8') as input_file, open(output_file_name, 'w', encoding='utf-8') as output_file:
        line_number = 0

        for line in input_file:
            line_number += 1

            # Write the current line to the output file
            output_file.write(line)

            # Check if the line number is divisible by 3, and if so, insert a blank line
            if line_number % 3 == 0:
                output_file.write('\n')

    print(f"Inserted blank lines in {input_file_name} and saved the result to {output_file_name}")


Inserted blank lines in removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt and saved the result to inserted_removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt
Inserted blank lines in removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt and saved the result to inserted_removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt
Inserted blank lines in removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt and saved the result to inserted_removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt
Inserted blank lines in removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt and saved the result to inserted_removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt


removing extra word

timeline

In [9]:
import glob
import os

def copy_and_replace_srt_lines(input_file1_pattern, input_file2_pattern, output_directory):
    input_files1 = glob.glob(input_file1_pattern)
    input_files2 = glob.glob(input_file2_pattern)

    if not input_files1:
        print(f"No matching files found for {input_file1_pattern}")
        return

    for input_file1 in input_files1:
        input_file1_basename = os.path.basename(input_file1)
        matching_input_file2 = None

        for input_file2 in input_files2:
            input_file2_basename = os.path.basename(input_file2)

            if input_file1_basename.endswith(input_file2_basename):
                matching_input_file2 = input_file2
                break

        if matching_input_file2:
            with open(input_file1, 'r', encoding='utf-8') as input_srt1:
                lines1 = input_srt1.readlines()

            with open(matching_input_file2, 'r', encoding='utf-8') as input_srt2:
                lines2 = input_srt2.readlines()

            for i in range(2, len(lines1), 4):
                if i < len(lines2):
                    lines2[i] = lines1[i]

            # Generate a unique output file name based on input files
            output_file_name = os.path.join(output_directory, f"updated_{input_file1_basename}")

            with open(output_file_name, 'w', encoding='utf-8') as output_srt2:
                output_srt2.writelines(lines2)

if __name__ == "__main__":
    input_file1_pattern = "inserted_removed_filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio*.srt"
    input_file2_pattern = "caption_of_youtube_audio*.srt"
    output_directory = ""  # Replace with the directory where you want to save the updated SRT files

    copy_and_replace_srt_lines(input_file1_pattern, input_file2_pattern, output_directory)


removes seconds word alternatively

In [72]:
import glob
import re

def process_srt_files(input_file_pattern, output_directory):
    input_files = glob.glob(input_file_pattern)

    for input_file in input_files:
        output_file = os.path.join(output_directory, f"processed_{os.path.basename(input_file)}")
        process_srt_file(input_file, output_file)

def process_srt_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    processed_captions = []
    remove_second_word = False

    for line in lines:
        line = line.strip()

        if line:
            if re.match(r'^\d+$', line):
                # Caption number
                caption_number = line
                processed_captions.append(caption_number)
            else:
                words = line.split()
                
                # Check if there are two words in the line
                if len(words) == 2:
                    # Remove the second word alternatively
                    if remove_second_word:
                        processed_line = words[0]
                        remove_second_word = False
                    else:
                        processed_line = line
                        remove_second_word = True
                else:
                    processed_line = line
                
                # Append the processed line
                processed_captions.append(processed_line)

    # Write the processed captions to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write('\n'.join(processed_captions))

if __name__ == "__main__":
    input_file_pattern = "updated_caption_of_youtube_audio*.srt"  # Pattern to match the input SRT files
    output_directory = ""  # Replace with the directory where you want to save the processed SRT files
    process_srt_files(input_file_pattern, output_directory)


adds spaces


In [73]:
import glob

def insert_blank_lines(input_file_pattern, output_directory):
    input_files = glob.glob(input_file_pattern)

    for input_file_name in input_files:
        output_file_name = os.path.join(output_directory, f"with_blank_lines_{os.path.basename(input_file_name)}")

        with open(input_file_name, 'r', encoding='utf-8') as input_file, open(output_file_name, 'w', encoding='utf-8') as output_file:
            line_number = 0

            for line in input_file:
                line_number += 1

                # Write the current line to the output file
                output_file.write(line)

                # Check if the line number is divisible by 3, and if so, insert a blank line
                if line_number % 3 == 0:
                    output_file.write('\n')

if __name__ == "__main__":
    input_file_pattern = "processed_updated_caption_of_youtube_audio*.srt"  # Pattern to match the input SRT files
    output_directory = ""  # Replace with the directory where you want to save the processed SRT files
    insert_blank_lines(input_file_pattern, output_directory)


The Timestamp Function


In [74]:
import os
import glob

def modify_srt_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as input_srt:
        lines = input_srt.readlines()

    modified_lines = []
    captions = []

    for line in lines:
        if line.strip():
            captions.append(line.strip())
        else:
            if captions:
                modified_lines.append(captions)
            captions = []

    for i in range(len(modified_lines) - 1):
        captions = modified_lines[i]
        next_captions = modified_lines[i + 1]

        if len(captions) == 3 and len(next_captions) == 3:
            start_time = captions[1].split(" --> ")[0]
            end_time = next_captions[1].split(" --> ")[0]

            modified_lines[i][1] = f"{start_time} --> {end_time}"

    with open(output_file, 'w', encoding='utf-8') as output_srt:
        for captions in modified_lines:
            output_srt.write('\n'.join(captions) + '\n\n')  # Add an extra newline to separate captions

if __name__ == "__main__":
    input_files = glob.glob('with_blank_lines_processed_updated_caption_of_youtube_audio*.srt')
    for input_srt_file in input_files:
        output_srt_file = f'modified_{os.path.basename(input_srt_file)}'
        modify_srt_file(input_srt_file, output_srt_file)
        print(f"Modified SRT file saved to {output_srt_file}")


Modified SRT file saved to modified_with_blank_lines_processed_updated_caption_of_youtube_audio1.srt
Modified SRT file saved to modified_with_blank_lines_processed_updated_caption_of_youtube_audio2.srt
Modified SRT file saved to modified_with_blank_lines_processed_updated_caption_of_youtube_audio3.srt
Modified SRT file saved to modified_with_blank_lines_processed_updated_caption_of_youtube_audio4.srt


word splitting 33 words

In [75]:
import os
import glob

def split_line_by_letter_count(text, letter_count):
    words = text.split()
    lines = []
    current_line = []
    current_line_letter_count = 0

    for word in words:
        if current_line_letter_count + len(word) <= letter_count:
            current_line.append(word)
            current_line_letter_count += len(word)
        else:
            lines.append(' '.join(current_line))
            current_line = [word]
            current_line_letter_count = len(word)

    if current_line:
        lines.append(' '.join(current_line))

    return lines

def insert_line_breaks(input_srt_file, output_folder, output_index, max_letters_per_line=33):
    with open(input_srt_file, 'r', encoding='utf-8') as file:
        srt_content = file.read()

    modified_srt = ""
    captions = srt_content.strip().split('\n\n')  # Split captions
    for caption in captions:
        lines = caption.split('\n')
        if len(lines) >= 3:
            caption_number, time_info, text = lines[0], lines[1], lines[2:]
            text = ' '.join(text)  # Combine text lines into one string
            lines = split_line_by_letter_count(text, max_letters_per_line)
            new_lines = [caption_number, time_info] + lines
            modified_srt += '\n'.join(new_lines) + '\n\n'
        else:
            modified_srt += caption + '\n\n'  # Retain captions with less than 3 lines

    output_file = f'caption_of_youtube_audio{output_index}.srt'
    output_srt_file = os.path.join(output_folder, output_file)

    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.write(modified_srt)

if __name__ == '__main__':
    input_files = glob.glob('modified_with_blank_lines_processed_updated_caption_of_youtube_audio*.srt')
    output_folder = r'D:\AfterEffectsBot\Assets\captions'
    output_index = 1
    for input_srt_file in input_files:
        insert_line_breaks(input_srt_file, output_folder, output_index)
        output_index += 1


In [76]:
import os

# Specify the folder containing the files
folder = 'D:\Youtube Bot Audio'  # Replace with your actual path

# List all SRT files in the folder
srt_files = [file for file in os.listdir(folder) if file.lower().endswith('.srt')]

# Delete SRT files
for srt_file in srt_files:
    srt_file_path = os.path.join(folder, srt_file)
    os.remove(srt_file_path)
    print(f"SRT file '{srt_file}' has been deleted.")

# List all files starting with "youtube_audio" in the folder
youtube_audio_files = [file for file in os.listdir(folder) if file.startswith('youtube_audio')]

# Delete files starting with "youtube_audio"
for youtube_audio_file in youtube_audio_files:
    youtube_audio_file_path = os.path.join(folder, youtube_audio_file)
    os.remove(youtube_audio_file_path)
    print(f"File '{youtube_audio_file}' has been deleted.")


SRT file 'caption_of_youtube_audio1.srt' has been deleted.
SRT file 'caption_of_youtube_audio2.srt' has been deleted.
SRT file 'caption_of_youtube_audio3.srt' has been deleted.
SRT file 'caption_of_youtube_audio4.srt' has been deleted.
SRT file 'duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt' has been deleted.
SRT file 'duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt' has been deleted.
SRT file 'duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt' has been deleted.
SRT file 'duplicated_output_combined_with_timestamps_caption_of_youtube_audio4.srt' has been deleted.
SRT file 'filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio1.srt' has been deleted.
SRT file 'filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio2.srt' has been deleted.
SRT file 'filtered_duplicated_output_combined_with_timestamps_caption_of_youtube_audio3.srt' has been deleted.
SRT file 'filtered_dupl

In [77]:
import subprocess

# Specify the path to the text file
file_path = "most_viewed_articles.txt"  # Replace with the actual path

try:
    subprocess.Popen(["notepad.exe", file_path])
except FileNotFoundError:
    print(f"File not found at '{file_path}'.")
except Exception as e:
    print(f"An error occurred: {e}")


import os
import glob

def insert_line_breaks(input_srt_file, output_folder, output_index):
    with open(input_srt_file, 'r', encoding='utf-8') as file:
        srt_content = file.read()

    modified_srt = ""
    captions = srt_content.strip().split('\n\n')  # Split captions
    for caption in captions:
        lines = caption.split('\n')
        if len(lines) >= 3:
            caption_number, time_info, text = lines[0], lines[1], lines[2:]
            words = text[0].split()  # Split the text into words
            first_line = ' '.join(words[:8])  # First line with 8 words
            if len(words) > 8:
                second_line = ' '.join(words[8:17])  # Second line with 9 words if available
                third_line = ' '.join(words[17:])  # Third line with the remaining words
            else:
                second_line = ' '.join(words[8:])  # Use the remaining words for the second line
                third_line = ''  # No third line
            new_lines = [caption_number, time_info, first_line, second_line, third_line] + text[1:]  # Retain other lines
            modified_srt += '\n'.join(new_lines) + '\n\n'
        else:
            modified_srt += caption + '\n\n'  # Retain captions with less than 3 lines

    output_file = f'caption_of_youtube_audio{output_index}.srt'
    output_srt_file = os.path.join(output_folder, output_file)

    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.write(modified_srt)

if __name__ == '__main__':
    input_files = glob.glob('modified_output_combined_with_timestamps_caption_of_youtube_audio*.srt')
    output_folder = r'D:\AfterEffectsBot\Assets\\captions'
    output_index = 1
    for input_srt_file in input_files:
        insert_line_breaks(input_srt_file, output_folder, output_index)
        output_index += 1


import os
import glob

def combine_words_to_caption(input_srt_file, output_srt_file):
    with open(input_srt_file, 'r', encoding='utf-8') as file:
        srt_content = file.read().splitlines()

    combined_srt = []
    words = []
    total_word_count = 0
    timestamps = []
    caption_index = 1

    def add_caption():
        nonlocal combined_srt, caption_index, timestamps, words
        if timestamps and len(timestamps) > 0:
            combined_srt.append(str(caption_index))
            if len(timestamps) == 1:
                combined_srt.append(timestamps[0])
            else:
                start_time, end_time = timestamps[0].split(" --> ")
                combined_srt.append(f"{start_time} --> {end_time}")
            combined_srt.append(" ".join(words))
            combined_srt.append("")
            caption_index += 1
        timestamps = []

    for line in srt_content:
        line = line.strip()
        if line.isdigit():
            add_caption()
        elif "-->" in line:
            timestamps.append(line)
        elif line:
            words.extend(line.split())
            total_word_count += len(line.split())

            while total_word_count >= 25:
                add_caption()
                words = words[25:]
                total_word_count = len(words)

    add_caption()

    with open(output_srt_file, 'w', encoding='utf-8') as file:
        file.write("\n".join(combined_srt))

if __name__ == '__main__':
    input_files = glob.glob('caption_of_youtube_audio*.srt')
    for input_srt_file in input_files:
        output_srt_file = f'output_combined_with_timestamps_{os.path.basename(input_srt_file)}'
        combine_words_to_caption(input_srt_file, output_srt_file)
